In [ ]:
pip install contractions

In [ ]:
pip install word2number

In [ ]:
pip install labMTsimple

In [ ]:
pip install marisa-trie

In [ ]:
pip install nameparser

In [ ]:
pip install allennlp

In [ ]:
pip install allennlp-models

In [ ]:
# Import Statements
import os
import csv
import math
import numpy as np
import pandas as pd
import math
import codecs
import re
import matplotlib.pyplot as plt
import contractions
import itertools
import spacy
from spacy.lang.en import English
from scipy.sparse import lil_matrix
from labMTsimple.speedy import LabMT
from word2number import w2n
from allennlp.predictors.predictor import Predictor
from nameparser import HumanName
import nltk
from nltk.corpus import names

In [ ]:
# Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
nltk.download('all')

In [ ]:
# Getting a corpus with male and female names from NLTK
male_names_dataset = list(set(names.words('male.txt')))
female_names_dataset = list(set(names.words("female.txt")))

# 1. Function to get the title of the book:

In [ ]:
def get_title(book):

    # Getting the Title of the book
    pattern1 = "Title:\s(.*)"
    match = re.search(pattern1, book.read())
    title = match.group(1)

    return title.strip()

# 2. Function to remove the first and last part of the book (Unnecessary):

In [ ]:
def clean_end_first(book, filename):

    # Removing the first part of the book
    pattern2 = "\*\*\*\s?START.*?\s?\*\*\*"
    book_clean = re.sub("^(?s:.*?)" + pattern2, " ", book.read(), count = 1)

    # Removing the last part of the book
    pattern3 = "\*\*\*\s?END.*?\s?\*\*\*"
    book_clean = re.sub(pattern3 + "(?s:.*)", " ", book_clean, count = 1)

    # Removing [Illustrations: ...] from the books
    pattern4 = "\[Illustration:(.*)\]"
    book_clean = re.sub(pattern4, "", book_clean)

    # Removing the text "The End" and others from the book
    pattern5 = "(?i)the end|end of the project gutenberg ebook of(.*)|end of project gutenberg's(.*)"
    book_clean = re.sub(pattern5, "", book_clean)

    # On Manual inspection, these books had the list of all books in the Wizard of Oz Series
    # This Regex will be used to remove that
    if filename == "8. Tik-Tok of Oz.txt" or filename == "9. The Scarecrow of Oz.txt" or filename == "10. Rinkitink in Oz.txt" or filename == "12. The Tin Woodman of Oz.txt" or filename == "14. Glinda of Oz.txt":
        pattern6 = "(The Wonderful Oz Books|THE FAMOUS OZ BOOKS)"
        book_clean = re.sub(pattern6 + "(?s:.*)", " ", book_clean)

    return book_clean

# 3. Function to Find all the Chapter Titles in the book:

In [ ]:
def chapter_titles(book, filename):
    # Finding all the chapter names in the book
    dictTOC = {}
    chapter_title = []

    if filename == "3. Ozma of Oz.txt" or filename == "4. Dorothy and the Wizard in Oz.txt" or filename == "5. The Road to Oz.txt" or filename == "13. The Magic of Oz.txt":
        # Pattern seen in Book 3, 4, 5, 13
        pattern7 = re.findall("(\d+)(\.)(\s+)(.*)", book)

        for i in range(len(pattern7)):
            title = pattern7[i][0] + pattern7[i][1] + pattern7[i][2] + pattern7[i][3]
            if len(pattern7[i][0]) <= 2:
                dictTOC[pattern7[i][0]] = title.strip()
    else:
        pattern8 = re.findall("(Chapter)(.*)", book)

        for i in range(len(pattern8)):
            title = pattern8[i][0] + pattern8[i][1]
            key_val = w2n.word_to_num(pattern8[i][1])
            dictTOC[key_val] = title.strip()

    for key in dictTOC:
        number = int(key) - 1
        chapter_title.insert(number, dictTOC[key])

    return chapter_title

# 4. Getting the text for each chapter in the book:

In [ ]:
def matching_chapter_titles(chapter_title, book_clean, filename, val):

    matches = re.finditer(re.escape(chapter_title), book_clean)
    count = 0
    temp = list(matches)
    length = len(temp)

    for i in range(length):
        if length == 2:
            match_index = temp[1].span()[val]
        else:
            match_index = temp[0].span()[val]

    return match_index

In [ ]:
def chapter_chunck(chapter_titles, book_clean, filename):

    chapter_chuncks = []

    for i in range(len(chapter_titles)):

        # Read till the next chapter title
        if i != (len(chapter_titles) - 1):

            # Getting the chapter title to start and end with
            chapter_title = chapter_titles[i]
            next_chapter_title = chapter_titles[i + 1]

            # Finding the positions in the text
            # This will get the chapter chunck

            # Start position
            end_of_start_chapter_title_index = matching_chapter_titles(chapter_title, book_clean, filename, 1)

            # End Position
            start_of_next_chapter_title_index = matching_chapter_titles(next_chapter_title, book_clean, filename, 0)

            # Getting the chapter chuncks and putting it into a list
            chapter_chuncks.append(book_clean[end_of_start_chapter_title_index:start_of_next_chapter_title_index])

        # Read till the end
        else:
            # This is done for the last chapter, where we will read till the end of the book

            chapter_title = chapter_titles[i]
            end_of_start_chapter_title_index = matching_chapter_titles(chapter_title, book_clean, filename, 1)
            chapter_chuncks.append(book_clean[end_of_start_chapter_title_index:])

    return chapter_chuncks

# 5. Function to remove paragraph spacing and weird line breaks:

In [ ]:
def remove_spacing_line_breaks(chapter_chuncks):

    # Each paragraph is separated using \r\n\r\n
    # Each new line is created using \r\n
    # Both of these will now be replaced using \s
    # Dealing with contractions as well

    chapter_chuncks_clean = []

    for chapter in chapter_chuncks:
        chapter = chapter.strip()
        chapter = chapter.replace("\r\n", " ")
        chapter = chapter.replace("\r\n\r\n", " ")
        chapter = contractions.fix(chapter)
        chapter_chuncks_clean.append(chapter)

    return chapter_chuncks_clean

# 6. Function to perform Sentence Tokenisation:

In [ ]:
def sent_tokenise(chapter_chuncks_clean):
    # Using SpaCy to perform sentence Tokenization

    sent_tokenizer = spacy.load("en_core_web_sm")
    all_sents_in_each_chapter = []

    for chapter in chapter_chuncks_clean:
        sent_chapter = []
        sentences = sent_tokenizer(chapter)

        for sent in sentences.sents:
            sent_chapter.append(sent)

        all_sents_in_each_chapter.append(sent_chapter)

    return all_sents_in_each_chapter

# 7. Function to perform Word Tokenisation:

In [ ]:
def word_tokenise(chapter_chuncks_clean):
    # Using SpaCy to perform word Tokenization
    # Loading the English Tokenizer

    word_tokenizer = English()
    all_words_in_each_chapter = []

    for chapter in chapter_chuncks_clean:
        words_chapter = []
        words = word_tokenizer(chapter.strip())
        for word in words:
            if not word.is_punct and len(word) != 0 and word.text.strip() != "":
                words_chapter.append(word.text.lower())
        all_words_in_each_chapter.append(words_chapter)

    return all_words_in_each_chapter

# 8. Named Entity Recognition (NER) using AllenNLP:

In [ ]:
# NER model
ner_model = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo.2021-02-12.tar.gz")

In [ ]:
def NER(all_sents_list, title):

  PER_tag = []
  flag = 0
  sentence_id = 0
  ner_path = "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/" + title + " - NER.csv"

  for sent in all_sents_list:

    person = ""
    sentence_id += 1
    print(sent)

    if sentence_id >= 1829:

      outcome = ner_model.predict(str(sent))

      for tag, word in zip(outcome["tags"], outcome["words"]):
        if tag == "B-PER":
          row = []
          person = word
          flag = 1
          continue

        elif tag == "U-PER":
          row = []
          person = word

          row_to_append = [person, sentence_id]
          with open(ner_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row_to_append)
          PER_tag.append(row_to_append)

        while flag == 1:
          if tag == "I-PER":
            person = person + " " + word
            break
          elif tag == "L-PER":
            person = person + " " + word
            flag = 0

            row_to_append = [person, sentence_id]
            with open(ner_path, mode='a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow(row_to_append)
            PER_tag.append(row_to_append)

          else:
            flag = 0

            row_to_append = [person, sentence_id]
            with open(ner_path, mode='a', newline='') as file:
              writer = csv.writer(file)
              writer.writerow(row_to_append)
            PER_tag.append(row_to_append)

  return PER_tag

In [ ]:
# Reading the Named Entity Recognition (NER) information from the CSV file
def NER_read_csv(title):

  ner_path = "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Named Entities/" + title + " - NER.csv"

  dataFrame = pd.read_csv(ner_path, skiprows = [1])
  dataFrame = dataFrame.iloc[:, :2]
  list_of_names = []

  for index, row in dataFrame.iterrows():
    row = row.to_dict()
    # Append the dictionary to the list
    list_of_names.append(row)

  return list_of_names

# 9. Function to get information related to names such as title, firstname, middlename, lastname, suffix and nickname (Done using NameParser):

In [ ]:
def get_name_info(list_of_names):

  name_info = []

  for element in list_of_names:
    name = HumanName(element["name"])
    name = name.as_dict()
    name_info.append(name)

  return name_info

# 10. Getting the characters in the book:

In [ ]:
def get_characters(list_of_names):

  characters = []

  for name in list_of_names:
    characters.append(name["name"])

  characters = list(set(characters))

  return characters

# 11. Corrections to some of the Character names:

In [ ]:
def correction_function(list_of_names, title):

  list_of_names_corrected = list_of_names[:]

  for name in list_of_names_corrected:

    if title == "Ozma of Oz":

      if name["name"] == "Jin":
        name["name"] = "Jin-jur"

      elif name["name"] == "Nome":
        name["name"] = "Nome King"

      elif name["name"] == "ma":
        name["name"] = "Oz-ma"

    elif title == "Dorothy and the Wizard in Oz":

      if name["name"] == "Bug" or name["name"] == "Woggle" or name["name"] == "T.E.":
        name["name"] = "Woggle-Bug"

      elif name["name"] == "Anu":
        name["name"] = "Overman-Anu"

      elif name["name"] == "Jim the Cab":
        name["name"] = "Jim the Cab-horse"

      elif name["name"] == "Tik":
        name["name"] = "Tik-tok"

    elif title == "The Road to Oz":

      if name["name"] == "Braided":
        name["name"] = "Braided Man"

      elif name["name"] == "Kik":
        name["name"] = "Kik-a-bray"

      elif name["name"] == "Tok":
        name["name"] = "Tok-Tok"

      elif name["name"] == "H. M. Wogglebug":
        name["name"] = "Wogglebug"

      elif name["name"] == "Ix":
        name["name"] = "Queen Zixi of Ix"

      elif name["name"] == "Tik":
        name["name"] = "Tik-tok"

      elif name["name"] == "Bug" or name["name"] == "H. M.":
        name["name"] = "Woggle-Bug"

      elif name["name"] == "Bright":
        name["name"] = "Button-Bright"

      elif name["name"] == "Nome":
        name["name"] = "Nome King"

    elif title == "The Emerald City of Oz":

      if name["name"] == "Scarecrow Ozma":
        name["name"] = "Scarecrow"

      elif name["name"] == "H. M. Wogglebug":
        name["name"] = "Wogglebug"

      elif name["name"] == "Over":
        name["name"] = "Mr. Over"

      elif name["name"] == "Uncle Henry--":
        name["name"] = "Uncle Henry"

      elif name["name"] == "Aunt" or name["name"] == "Them":
        name["name"] = "Aunt Them"

    elif title == "The Patchwork Girl of Oz":

      if name["name"] == "Tottenhot Waits":
        name["name"] = "Tottenhot"

      elif name["name"] == "Woggle":
        name["name"] = "Woggle-Bug"

      elif name["name"] == "Tok":
        name["name"] = "Tik-Tok"

      elif name["name"] == "Kroo" or name["name"] == "Krizzle":
        name["name"] = "Krizzle-Kroo"

      elif name["name"] == "Ozma'll":
        name["name"] = "Ozma"

      elif name["name"] == "Wise Scarecrow":
        name["name"] = "Scarecrow"

      elif name["name"] == "I. Toto":
        name["name"] = "Toto"

      elif name["name"] == "Journey Ojo":
        name["name"] = "Ojo"

      elif name["name"] == "Dorothy Dorothy Gale":
        name["name"] = "Dorothy Gale"

      elif name["name"] == "een Ojo":
        name["name"] = "Ojo"

    elif title == "Tik-Tok of Oz":

      if name["name"] == "Tititi":
        name["name"] = "Tititi-Hoocho"

      elif name["name"] == "Eared Hearer":
        name["name"] = "Long-Eared Hearer"

      elif name["name"] == "Tok":
        name["name"] = "Tik-Tok"

      elif name["name"] == "Nome":
        name["name"] = "Nome King"

      elif name["name"] == "Naughty Nome Shaggy Man":
        name["name"] = "Shaggy Man"

    elif title == "The Scarecrow of Oz":

      if name["name"] == "Trot!—is":
        name["name"] = "Trot"

      elif name["name"] == "Tok":
        name["name"] = "Tik-Tok"

      elif name["name"] == "Googly":
        name["name"] = "Googly-Goo"

      elif name["name"] == "Bumpy":
        name["name"] = "Bumpy Man"

      elif name["name"] == "Hopper":
        name["name"] = "Grass-Hopper"

      elif name["name"] == "Button":
        name["name"] = "Button-Bright"

    elif title == "Rinkitink in Oz":

      if name["name"] == "Zella Goes" or name["name"] == "Zella Saves":
        name["name"] = "Zella"

      elif name["name"] == "Kitticut?--a":
        name["name"] = "Kitticut"

      elif name["name"] == "Said Rinkitink":
        name["name"] = "Rinkitink"

      elif name["name"] == "Inga Parts":
        name["name"] = "Inga"

      elif name["name"] == "Wooden Sawhorse":
        name["name"] = "Sawhorse"

      elif name["name"] == "Nome":
        name["name"] = "Nome King"

      elif name["name"] == "Tiny Trot":
        name["name"] = "Trot"

    elif title == "The Tin Woodman of Oz":

      if name["name"] == "Klip":
        name["name"] = "Ku-Klip"

      elif name["name"] == "Yookoohoo Woot":
        name["name"] = "Woot"

      elif name["name"] == "Brown":
        name["name"] = "Brown Bear"

      elif name["name"] == "Tik":
        name["name"] = "Tik-tok"

      elif name["name"] == "Wooden Sawhorse":
        name["name"] = "Sawhorse"

      elif name["name"] == "Bright":
        name["name"] = "Button-Bright"

      elif name["name"] == "Tiny Trot":
        name["name"] = "Trot"

    elif title == "The Magic of Oz":

      if name["name"] == "Bright":
        name["name"] = "Button-Bright"

      elif name["name"] == "Lucky":
        name["name"] = "Ojo the Lucky"

      elif name["name"] == "Tik":
        name["name"] = "Tik-Tok"

      elif name["name"] == "O Rango":
        name["name"] = "Rango the Gray Ape"

      elif name["name"] == "OZMA" or name["name"] == "Ozma Has":
        name["name"] = "Ozma"

      elif name["name"] == "H. M. Wogglebug":
        name["name"] = "Wogglebug"

      elif name["name"] == "Wooden Sawhorse":
        name["name"] = "Sawhorse"

      elif name["name"] == "Eag":
        name["name"] = "Li-Mon-Eag"

    elif title == "Glinda of Oz":

      if name["name"] == "Su":
        name["name"] = "Su-dic"

      elif name["name"] == "Coo":
        name["name"] = "Coo-ee-oh"

      elif name["name"] == "Great Sorceress Betsy":
        name["name"] = "Betsy"

      elif name["name"] == "Lucky":
        name["name"] = "Ojo the Lucky"

      elif name["name"] == "Magic Isle Ozma":
        name["name"] = "Ozma"

      elif name["name"] == "Tok":
        name["name"] = "Tik-Tok"

      elif name["name"] == "H. M. Wogglebug":
        name["name"] = "Wogglebug"

      elif name["name"] == "Nome":
        name["name"] = "Nome King"

      elif name["name"] == "Flathead Su":
        name["name"] = "Flathead Su-dic"

      elif name["name"] == "Betsy Glinda":
        name["name"] = "Glinda"

  return list_of_names_corrected

# 12. Function to get the gender of the characters:

In [ ]:
# Initialisation
male_titles = ["Uncle", "Mr", "Prince", "King", "Emperor", "Colonel", "Judge", "Mister", "Dr", "Professor", "Brother", "Private",
               "General", "Captain", "Chief", "Wizard", "Mr.", "Royal", "Emperor"]
female_titles = ["Mother", "Princess", "Queen", "Aunt", "Miss", "Empress", "Dame", "Mrs", "Lady", "Witch", "Madam", "Highness"]
female_pronouns = ["she", "her", "hers", "herself", "Princess", "Queen", "niece", "girl"]
male_pronouns = ["he", "him", "himself", "Prince", "King", "nephew", "boy"]

In [ ]:
# Reading the text file to get additional male names
with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Female names.txt", "r") as f:
    female_name = f.readlines()
female_name = [name.strip() for name in female_name]

In [ ]:
# Reading the text file to get additional female names
with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Male names.txt", "r") as f:
     male_name = f.readlines()
male_name = [name.strip() for name in male_name]

In [ ]:
def gender_identification(total_names_info, title):

  detected_name = []

  for i in range(len(total_names_info)):

    gender = ""

    # The Title provides useful information for gender identification
    if len(total_names_info[i]["title"]) > 0:
      # Check to see if they are in the list of male or female titles
      if total_names_info[i]["title"] in male_titles:
        gender = "male"

      elif total_names_info[i]["title"] in female_titles:
        gender = "female"

      total_names_info[i]["gender"] = gender

      # All the other character names with the same will be updated as having the same gender
      for j in range(i + 1, len(total_names_info)):
        if total_names_info[i]["name"] == total_names_info[j]["name"]:
          total_names_info[j]["gender"] = gender

    # The Title does not provide any useful information
    # Using the First name for gender idenfication
    elif len(total_names_info[i]["title"]) == 0 and len(total_names_info[i]["gender"]) == 0 and total_names_info[i]["first"] in male_names_dataset or total_names_info[i]["first"] in female_names_dataset or total_names_info[i]["first"] in male_name or total_names_info[i]["first"] in female_name:

      # Check to see if the names are in the dataset
      if total_names_info[i]["first"] in female_name:
        gender = "female"
      elif total_names_info[i]["first"] in male_name:
        gender = "male"
      elif total_names_info[i]["first"] in male_names_dataset:
        gender = "male"
      elif total_names_info[i]["first"] in female_names_dataset:
        gender = "female"

      total_names_info[i]["gender"] = gender

      # All the other character names with the same will be updated as having the same gender
      for j in range(i + 1, len(total_names_info)):
        if total_names_info[i]["name"] == total_names_info[j]["name"]:
          total_names_info[j]["gender"] = gender

  for k in range(len(total_names_info)):

    # The Title does not provide any useful information
    # The First name does not provide any useful information
    # Using the context of the sentence to get the information
    if len(total_names_info[k]["gender"]) == 0:

      name = total_names_info[k]["name"]
      detected_name.append(name)

  for d_name in detected_name:
    print(d_name)

  unknown_names_gender_context(detected_name, total_names_info)

  return total_names_info

In [ ]:
def unknown_names_gender_context(detected_name, total_names_info):

  nlp = English()
  word_tokenizer = nlp.tokenizer
  detected_name = list(set(detected_name))

  for i in range(len(detected_name)):
    if "-" in detected_name[i]:
      detected_name[i] = re.sub("-\s", "", detected_name[i])

  for i in range(len(total_names_info)):

    if len(total_names_info[i]["gender"]) == 0:

      # Sentence associated with the character
      tokenised_sent = word_tokenizer(str(all_sents_list[(total_names_info[i]["sentence_id"]) - 1]))

      # Word Tokenisation
      words = []
      for word in tokenised_sent:
        if not word.is_punct and len(word) != 0 and word.text.strip() != "":
          words.append(word.text)

      # If the name has more then one word, then we will find the index of the last word in the name
      # We will get the 5 words after that
      if total_names_info[i]["name"].count(" ") >= 1:

        name_split = total_names_info[i]["name"].split()
        length = len(name_split)
        name = name_split[length - 1]
      else:

        name = total_names_info[i]["name"]

      try:
        # Initialisation
        index = words.index(name)
        female_pronouns_counter = 0
        male_pronouns_counter = 0
        right_three_words = []
        left_three_words = []

        # Getting the context (5 words after the occurence of the name)
        if len(words) - (index + 1) == 1 and len(words) - (index + 1) == 2 and len(words) - (index + 1) == 3 and len(words) - (index + 1) == 4 and len(words) - (index + 1) == 5:
        # Read till the end
          right_three_words = words[index + 1:]
        else:
        # Read the next 5 words after the occurence of a character
          right_three_words = words[index + 1:index + 6]

        # Getting the context (5 words before the occurence of the name)
        if index == 0:
          pass
        elif index == 1 or index == 2 or index == 3 or index == 4 or index == 5:
          left_three_words = words[:index]
        else:
          left_three_words = words[index - 5:index]

        # Putting the context into a single list
        trigram = left_three_words + right_three_words

        # Counter to check whether the male or female pronouns are present
        for t in trigram:
          if t in female_pronouns:
            female_pronouns_counter += 1
          elif t in male_pronouns:
            male_pronouns_counter += 1

        # Whichever pronoun has a higher count, the character will be assigned that gender
        if female_pronouns_counter > male_pronouns_counter:
          gender = "female"
        elif male_pronouns_counter > female_pronouns_counter:
          gender = "male"
        else:
          gender = "unknown"

        if gender == "unknown":
          total_names_info[i]["gender"] = gender

        else:
          total_names_info[i]["gender"] = gender

          # All the other character names with the same will be updated as having the same gender
          for j in range(i + 1, len(total_names_info)):
            if total_names_info[i]["name"] == total_names_info[j]["name"]:
              total_names_info[j]["gender"] = gender

      except:
        pass

# 12. Function to store the characters and all the related information:

In [ ]:
def store_all_info(total_names_info, title):
  # Inserting all the information related to the characters into a CSV file for each book
  # Headings
  headings = ["name", "sentence_id", "title", "first", "middle", "last", "suffix", "nickname", "gender"]

  # Inserting character information into the csv files
  for i in range(len(total_names_info)):
    row_to_insert = []

    row_to_insert.append(total_names_info[i]["name"])
    row_to_insert.append(total_names_info[i]["sentence_id"])

    for j in range(2, len(headings)):
      temp = headings[j]
      if len(total_names_info[i][temp]) > 0:
        row_to_insert.append(total_names_info[i][temp])
      else:
        row_to_insert.append("none")

    with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Characters Information/" + title + " - All Info.csv", "a", newline = "") as f:
        write = csv.writer(f)
        write.writerow(row_to_insert)

# 13. Matching mulitple entities and their nicknames:

In [ ]:
def matching_algorithm(title):

  # Reading the CSV file and getting the list of characters
  dataFrame = pd.read_csv("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Characters Information/" + title + " - All Info.csv")
  character_names = dataFrame["name"].tolist()
  character_names = list(set(character_names))

  # Converting the DataFrame into a list
  total_info = dataFrame.to_dict(orient = "list")

  # Matching Algorithm
  all_entities = []

  for i in range(len(total_info["name"])):

    entity = []

    # Check to see if there exists names with title, first name and last name
    # This is the most complete form of the name
    if total_info["title"][i] != "none" and total_info["first"][i] != "none" and total_info["last"][i] != "none":
      name = total_info["name"][i]

      if len(all_entities) != 0:
        # Check to see if that name already exists
        if any(name in temp for temp in all_entities) or name in entity:
          continue
        else:
          entity.append(name)

      else:
        if name in entity:
          continue
        else:
          entity.append(name)

      # Looking for matches
      for j in range(i + 1, len(total_info["name"])):

        # Look for first and last names that match
        if total_info["first"][j] == total_info["first"][i] and total_info["last"][j] == total_info["last"][i] and total_info["gender"][j] == total_info["gender"][i]:

          name = total_info["name"][j]

          if len(all_entities) != 0:
            # Check to see if that name already exists
            if any(name in temp for temp in all_entities) or name in entity:
              continue
            else:
              entity.append(name)

          else:
            if name in entity:
              continue
            else:
              entity.append(name)

        # Look for first names that match
        elif total_info["first"][j] == total_info["first"][i] and total_info["gender"][j] == total_info["gender"][i]:

          name = total_info["name"][j]

          if len(all_entities) != 0:
            # Check to see if that name already exists
            if any(name in temp for temp in all_entities) or name in entity:
              continue
            else:
              entity.append(name)

          else:
            if name in entity:
              continue
            else:
              entity.append(name)

      all_entities.append(entity)

    # Check to see if there exists first and last name
    # This is the second most complete form of the name
    if total_info["first"][i] != "none" and total_info["last"][i] != "none":
      name = total_info["name"][i]

      if len(all_entities) != 0:
        # Check to see if that name already exists
        if any(name in temp for temp in all_entities) or name in entity:
          continue
        else:
          entity.append(name)

      else:
        if name in entity:
          continue
        else:
          entity.append(name)

      # Looking for matches
      for j in range(i + 1, len(total_info["name"])):
        # Look for first names that match
        if total_info["first"][j] == total_info["first"][i] and total_info["gender"][j] == total_info["gender"][i]:

          name = total_info["name"][j]

          if len(all_entities) != 0:
            # Check to see if that name already exists
            if any(name in temp for temp in all_entities) or name in entity:
              continue
            else:
              entity.append(name)

          else:
            if name in entity:
              continue
            else:
              entity.append(name)

      all_entities.append(entity)

    # Only names with first names
    if total_info["first"][i] != "none":
      name = total_info["name"][i]

      if len(all_entities) != 0:
        # Check to see if that name already exists
        if any(name in temp for temp in all_entities) or name in entity:
          continue
        else:
          entity.append(name)

      else:
        if name in entity:
          continue
        else:
          entity.append(name)

      # Looking for matches
      for j in range(i + 1, len(total_info["name"])):
        # Look for first names that match
        if total_info["first"][j] == total_info["first"][i] and total_info["gender"][j] == total_info["gender"][i]:

          name = total_info["name"][j]

          if len(all_entities) != 0:
            # Check to see if that name already exists
            if any(name in temp for temp in all_entities) or name in entity:
              continue
            else:
              entity.append(name)

          else:
            if name in entity:
              continue
            else:
              entity.append(name)

      all_entities.append(entity)

  # Characters that are found by the matching algorithm
  matched_char = [ele for subl in all_entities for ele in subl]

  for char in character_names:
    if char not in matched_char:
      all_entities.append([char])

  return all_entities

In [ ]:
def nicknames_and_matches(title, all_entities):

  nickname_list = []
  new_list = []
  final_entities = []

  # Reading the nicknames file
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Nicknames.txt", 'r') as f:
    for line in f:
      line = line.strip()
      words = line.split(',')
      nickname_list.append(words)

  # Getting all the nicknames for each of the matches
  for entity in all_entities:
    for character in entity:
      flag = 0
      for nicknames in nickname_list:
        if character in nicknames:
          flag += 1
          break

    if flag >= 1:
      new_list.append(entity + nicknames)
    elif flag == 0:
      new_list.append(entity)

  # Removing the duplicate elements from the inner list
  for l in new_list:
    final_entities.append(list(set(l)))

  # Removing duplicate inner lists
  sort_i_list = [frozenset(i_l) for i_l in final_entities]
  unique_entities_temp = set(sort_i_list)
  unique_entities = [list(i_s) for i_s in unique_entities_temp]

  return unique_entities

# 14. Entity Resolution and Top Characters:

In [ ]:
def entity_resolution(title):

  # Reading the matched entities text file
  matched_entities = []

  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Matched Characters and Nicknames/" + title + " - Matches.txt", 'r') as f:
    for line in f:
      line = line.strip()
      words = line.split(',')
      matched_entities.append(words)

  # Saving the longest name as the one to replace all the other names
  matched_entities_dict = []
  for item in matched_entities:

    if len(item) != 1:
      dictionary = {"name":"", "other": []}
      longest_name = max(item, key = len)

      dictionary["name"] = longest_name
      item.remove(longest_name)

      for character in item:
        dictionary["other"].append(character)

    elif len(item) == 1:
      dictionary = {"name":item[0], "other":[]}

    matched_entities_dict.append(dictionary)

  # Reading the Characters - All Info CSV file and reading only the names and the sentence_id
  dataFrame1 = pd.read_csv("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Characters Information/" + title + " - All Info.csv", skiprows = [1])
  dataFrame1 = dataFrame1.iloc[:, :2]
  list_of_names1 = []

  for index, row in dataFrame1.iterrows():
    row = row.to_dict()
    # Append the dictionary to the list
    list_of_names1.append(row)

  # Creating a CSV file with the characters and the resolved names
  for ele in list_of_names1:

    name = ele["name"]
    sentence_id = ele["sentence_id"]
    new_name = ""
    row_to_insert = []

    for entity in matched_entities_dict:
      if len(entity["other"]) == 0:
        if name in entity["name"]:
          new_name = entity["name"]
      else:
        if name in entity["other"]:
          new_name = entity["name"]
        elif name == entity["name"]:
          new_name = name

    row_to_insert.append(name)
    row_to_insert.append(sentence_id)
    row_to_insert.append(new_name)

    with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Entity Changes/" + title + " - Entity Changes.csv", "a", newline = "") as f:
      write = csv.writer(f)
      write.writerow(row_to_insert)


In [ ]:
def top_10_characters(title):

    # Getting the top 10 characters based on their occurences
    cols = ["name", "sentence_id", "new_name"]
    df = pd.read_csv("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Entity Changes/" + title + " - Entity Changes.csv", usecols = cols)
    unique_values = df["new_name"].value_counts()
    top_10_characters = unique_values.nlargest(10)
    top_characters = [(character, num_occurences) for character, num_occurences in zip(top_10_characters.index, top_10_characters.values)]

    # Saving this information in a text file
    with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Top 10 characters/" + title + " - Top 10 Characters.txt", "w", newline = "") as f:
      csv_writer = csv.writer(f)
      csv_writer.writerows(top_characters)

In [ ]:
def resolution(title, all_sents_list):

  # Reading the entity changes file
  dataFrame1 = pd.read_csv("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Entity Changes/" + title + " - Entity Changes.csv")
  dataFrame1 = dataFrame1.iloc[:, :3]
  list_of_names1 = []
  entity_changes_sents = all_sents_list.copy()

  for index, row in dataFrame1.iterrows():
    row = row.to_dict()
    # Append the dictionary to the list
    list_of_names1.append(row)

  # Resolving the entities into a single entity
  for item in list_of_names1:
    name = item["name"]
    sentence_index = item["sentence_id"] - 1
    new_name = item["new_name"]
    sentence = str(entity_changes_sents[sentence_index])
    entity_changes_sents[sentence_index] = sentence.replace(name, new_name)

  # Saving the original list of sentences to a text file
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Original Text - Sentences/" + title + " - Original.txt", 'w') as f:
    for sentence in all_sents_list:
      f.write(str(sentence) + '\n')

  # Saving the changed sentences to a text file
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/New text - After Character Resolution/" + title + " - Resolved.txt", 'w') as f:
    for sentence in entity_changes_sents:
      f.write(str(sentence) + '\n')


# 15. Coreference Resolution:

In [ ]:
# Loading the model
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

In [ ]:
def coreference_resolution(title):

  text_coref = []
  coreference_resolved_text = []
  window_size = 5
  sent_tokenizer = spacy.load("en_core_web_sm")

  # Reading the respective file
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/New text - After Character Resolution/" + title + " - Resolved.txt", 'r') as f:
    for line in f:
      text_coref.append(line.strip())

  for i in range(0, len(text_coref), 5):
    start_index = i
    end_index = i + window_size

    if end_index > len(text_coref):
      text = ' '.join(text_coref[start_index:])
    else:
      text = ' '.join(text_coref[start_index:end_index])

    coref_text = predictor.coref_resolved(text)

    # Getting back the sentences
    sentences = sent_tokenizer(coref_text)
    for sent in sentences.sents:
      coreference_resolved_text.append(sent)

  # Saving the information
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/New text - Coreference Resolution/" + title + " - Coref.txt", 'w') as f:
    for sentence in coreference_resolved_text:
      f.write(str(sentence) + '\n')

# 16. Semantic Role Labelling:

In [ ]:
srl_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

In [ ]:
# This is for the Coference Resolution Text

def semantic_role_labelling_coref(title):

  coreference_resolved_text = []

  # Reading the coreference resolution text
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/New text - Coreference Resolution/" + title + " - Coref.txt", 'r') as f:
    for line in f:
      coreference_resolved_text.append(line.strip())

  for index, sentence in enumerate(coreference_resolved_text):
    sentence = contractions.fix(sentence)

    flag = 0
    outcome = srl_predictor.predict(sentence=sentence)
    for item in outcome["verbs"]:

      character_text= []
      tags = item["tags"]

      for j in range(len(tags)):

        if tags[j] == "B-ARG0":
          agent = outcome["words"][j]
          flag = 1
          continue

        while flag == 1:

          if tags[j] == "I-ARG0":
            agent = agent + " " + outcome["words"][j]
            break

          else:

            character_text.append(index)
            character_text.append(agent)

            # Writing the agents and the sentence index into the CSV file
            with open('/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Semantic Role Labelling - Coref/' + title + " - Semantic Role Labelling.csv", 'a', newline='') as csvf:
              csv_w = csv.writer(csvf)
              num_elements = len(character_text)
              for i in range(0, num_elements, 2):
                  row = character_text[i:i + 2]
                  csv_w.writerow(row)

            flag = 0

# 17. Getting the Character based sentences using the Window size:

In [ ]:
# Coreference Resolved Text
def character_based_sentences_coref(title):

  window_size = 10000
  num_of_chuncks = 100
  top_10_characters = []
  coreference_resolved_text = []

  # Reading the semantic role labelled file
  srl_column = ["sentence_index", "agent"]
  srl_df = pd.read_csv("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Semantic Role Labelling - Coref/" + title + " - Semantic Role Labelling.csv", usecols = srl_column)

  # Reading the top 3 characters
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Top 10 characters/" + title + " - Top 10 Characters.txt", 'r') as f:
    for line in f:
      character = line.strip().split(",")[0]
      top_10_characters.append(character)

  top_3_characters = top_10_characters.copy()

  # Reading the top 3 characters in each book
  if title == "The Road to Oz":
    top_3_characters = []
    top_3_characters.append(top_10_characters[0])
    top_3_characters.append(top_10_characters[2])
    top_3_characters.append(top_10_characters[3])

  elif title == "The Scarecrow of Oz":
    top_3_characters = top_3_characters[0:2]
    top_3_characters.append(top_10_characters[3])

  else:
    top_3_characters = top_3_characters[0:3]

  # Reading the word list along with the index values to get the window size
  columns_to_read = ["words", "index_position", "sentence_id"]
  window_df = pd.read_csv("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/DataFrame to get the Window - Coref/" + title + " - Words.csv", usecols = columns_to_read, skiprows = [1])

  # Reading the Coreference Resolved text for the Character's Arc
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/New text - Coreference Resolution/" + title + " - Coref.txt", 'r') as f:
    for line in f:
      coreference_resolved_text.append(line.strip())

  step_size = int(math.floor(window_df.shape[0] - window_size) / (num_of_chuncks - 1))

  for character in top_3_characters:

    for i in range(num_of_chuncks):

      character_resolved_text = []
      row_to_append = []
      character_presence = []
      row = []

      # Getting the start and end sentence index positions
      index_start_word = i * step_size
      index_end_word = window_size + (i * step_size)

      start_word_row = window_df.loc[(window_df["index_position"] == index_start_word)]
      start_id = start_word_row.loc[start_word_row.index[0], "sentence_id"]
      start_sentence_index = start_id - 1

      end_word_row = window_df.loc[(window_df["index_position"] == index_end_word)]
      end_id = end_word_row.loc[end_word_row.index[0], "sentence_id"]
      end_sentence_index = end_id

      character_df = srl_df[(srl_df["sentence_index"] >= start_sentence_index) & (srl_df["sentence_index"] <= end_sentence_index) & (srl_df["agent"].str.contains(character_corrected))]
      character_text_list = character_df["sentence_index"].tolist()
      character_text_list = list(set(character_text_list))
      character_text_list_sorted = character_text_list.copy()
      character_text_list_sorted.sort()

      # Getting the text from the window size
      if i != num_of_chuncks - 1:
        text = coreference_resolved_text[start_sentence_index:end_sentence_index]

      else:
        text = coreference_resolved_text[start_sentence_index:]

      block_of_text = " ".join(list(map(str, text)))

      # Getting the character related text from the window size
      for index in character_text_list_sorted:
        character_resolved_text.append(coreference_resolved_text[index])

      character_block_text = " ".join(list(map(str, character_resolved_text)))

      # Append it to a list
      row_to_append.append(i + 1)
      row_to_append.append(block_of_text)
      row_to_append.append(len(text))
      row_to_append.append(character_block_text)
      row_to_append.append(len(character_resolved_text))

      # Adding it into a CSV file
      with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Character Text for Character Arc Generation - Coref/" + title + "/" + title + " - " + character + ".csv", mode = 'a', newline = '') as f:
        writer = csv.writer(f)
        writer.writerow(row_to_append)

# 18. Emotion Arc related functions:

In [ ]:
# Function to get the count of each word in the text

def word_freq(words_in_text):
    count_dict = {}

    for word in words_in_text:
        if word in count_dict:
            count_dict[word] += 1
        else:
            count_dict[word] = 1

    return count_dict

In [ ]:
# Stopper Function
# This function was taken directly from the Documentation written by the authors
# Referencing the author's work as there were issues with accessing this function
# Reference: Mitchell L. Kiley D. Danforth C.M. Reagan, A.J. and P.S. Dodds. The emotional arcs of stories are dominated by six basic shapes, 2016.

def stopper(tmpVec,score_list,word_list,stopVal=1.0,ignore=[],center=5.0):
    """Take a frequency vector, and 0 out the stop words.

    Will always remove the nig* words.

    Return the 0'ed vector."""

    ignoreWords = ["nigga","nigger","niggaz","niggas"];
    for word in ignore:
        ignoreWords.append(word)
    newVec = tmpVec
    for i in range(len(score_list)):
        if abs(score_list[i]-center) < stopVal:
            newVec[i] = 0
        if word_list[i] in ignoreWords:
            newVec[i] = 0

    return newVec

# 19. Generating the Character Arc of the Top 3 characters in each of the books:

In [ ]:
# Creating an instance of the LabMT class
LabMT = LabMT()

In [ ]:
def emotional_arc(title, path1, path2):

  num_of_chuncks = 100
  top_10_characters = []
  word_tokenizer = English()

  # Reading the top 3 characters
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Top 10 characters/" + title + " - Top 10 Characters.txt", 'r') as f:
    for line in f:
      character = line.strip().split(",")[0]
      top_10_characters.append(character)

  top_3_characters = top_10_characters.copy()

  # Reading the top 3 characters in each book
  if title == "The Road to Oz":
    top_3_characters = []
    top_3_characters.append(top_10_characters[0])
    top_3_characters.append(top_10_characters[2])
    top_3_characters.append(top_10_characters[3])

  elif title == "The Scarecrow of Oz":
    top_3_characters = top_3_characters[0:2]
    top_3_characters.append(top_10_characters[3])

  else:
    top_3_characters = top_3_characters[0:3]

  for character in top_3_characters:

    # Reading the respective file for the character based sentences
    character_sentences_dataframe = pd.read_csv(path1 + title + "/" + title + " - " + character + ".csv", skiprows = [1])

    freq_matrix = lil_matrix((num_of_chuncks, len(LabMT.scorelist)))

    for counter, block_of_text in enumerate(character_sentences_dataframe["character_text_in_window"]):

      block_of_text = str(block_of_text)

      # Word Tokenisation for the block of text
      all_words_in_sentences = []
      words = word_tokenizer(block_of_text.strip())
      for word in words:
        if not word.is_punct and len(word) != 0 and word.text.strip() != "":
          all_words_in_sentences.append(word.text.lower())

      word_freq_dict = word_freq(all_words_in_sentences)
      freq_matrix[counter, :] = LabMT.wordVecify(word_freq_dict)

    # Generating the emotion scores
    emotion_score = list(np.zeros((num_of_chuncks)))
    X = []

    for i in range(num_of_chuncks):
        X.append(i)

    for i in range(num_of_chuncks):

      # Convert each row of the sparse matrix representation into a list
      text_vec = freq_matrix[i, :].toarray().squeeze()

      stoppedVec = stopper(text_vec,LabMT.scorelist,LabMT.wordlist)

      # Getting the scores based on the frequency of the words appearance in the chunck
      emotion_score[i] = np.dot(LabMT.scorelist,stoppedVec)/sum(stoppedVec)

    emotion_score = [0 if math.isnan(score) else score for score in emotion_score]

    # Saving the emotion scores
    with open(path2 + title + "/" + title + " - " + character + ".csv", "a", newline = "") as csvf:
      csv_w = csv.writer(csvf)

      csv_w.writerow(["window_number", "emotion_score"])

      for ele1, ele2 in zip(X, emotion_score):
          csv_w.writerow([ele1, ele2])

# 20. Drawing the Arc's:

In [ ]:
# Comparing the character arcs for each book
def character_arc_compare(title, path1, path2):

  top_10_characters = []

  # Reading the top 3 characters
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Top 10 characters/" + title + " - Top 10 Characters.txt", 'r') as f:
    for line in f:
      character = line.strip().split(",")[0]
      top_10_characters.append(character)

  top_3_characters = top_10_characters.copy()

  # Reading the top 3 characters in each book
  if title == "The Road to Oz":
    top_3_characters = []
    top_3_characters.append(top_10_characters[0])
    top_3_characters.append(top_10_characters[2])
    top_3_characters.append(top_10_characters[3])

  elif title == "The Scarecrow of Oz":
    top_3_characters = top_3_characters[0:2]
    top_3_characters.append(top_10_characters[3])

  else:
    top_3_characters = top_3_characters[0:3]

  figure, ax = plt.subplots()

  for character in top_3_characters:

    X = []
    emotional_score = []

    with open(path1 + title + " - " + character + ".csv", "r") as csvfile:
      csvreader = csv.reader(csvfile)
      for row in csvreader:
        if len(row) >= 2:
          X.append(row[0])
          emotional_score.append(row[1])

    X = X[1:]
    emotional_score = emotional_score[1:]
    X = [int(x) for x in X]
    emotional_score = [float(x) for x in emotional_score]
    X = [x for x, score in zip(X, emotional_score) if score != 0]
    emotional_score = [score for score in emotional_score if score != 0]

    ax.plot(X, emotional_score, label = character)
    ax.legend()

  ax.set_xlabel("Percentage of Book")
  ax.set_ylabel("Emotional Score based on Valence")
  # ax.set_title(title, fontsize = 18)
  #plt.savefig("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Results - Plots/Character Arc Comparison/" + title + ".pdf")
  plt.show()
  print()
  print()

In [ ]:
# Plotting the individual character arcs
def individual_arcs(path, title):

  if title == "The Road to Oz":
    top_10_characters = []

    # Reading the top 3 characters
    with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Top 10 characters/" + title + " - Top 10 Characters.txt", 'r') as f:
      for line in f:
        character = line.strip().split(",")[0]
        top_10_characters.append(character)

    top_3_characters = top_10_characters.copy()

    # Reading the top 3 characters in each book
    if title == "The Road to Oz":
      top_3_characters = []
      top_3_characters.append(top_10_characters[0])
      top_3_characters.append(top_10_characters[2])
      top_3_characters.append(top_10_characters[3])

    elif title == "The Scarecrow of Oz":
      top_3_characters = top_3_characters[0:2]
      top_3_characters.append(top_10_characters[3])

    else:
      top_3_characters = top_3_characters[0:3]

    for character in top_3_characters:

      X = []
      emotional_score = []
      new_path = path + title + " - " + character + ".csv"

      with open(new_path, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
          if len(row) >= 2:
            X.append(row[0])
            emotional_score.append(row[1])

      X = X[1:]
      emotional_score = emotional_score[1:]
      X = [int(x) for x in X]
      emotional_score = [float(x) for x in emotional_score]
      X = [x for x, score in zip(X, emotional_score) if score != 0]
      emotional_score = [score for score in emotional_score if score != 0]

      plt.plot(X, emotional_score)
      plt.xlabel("Percentage of the Book")
      plt.ylabel("Emotional Score based on Valence")
      plt.title(title + " - " + character)
      plt.savefig("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Results - Plots/Character Arcs/" + title + "/" + title + " - " + character + " - Final.pdf")
      plt.show()
      print()
      print()


In [ ]:
# Comparing individual character arcs with the emotion arc of the narrative
def character_and_emotional_comparison_individual(title, path1, path2):

  top_10_characters = []
  X1 = []
  emotional_arc = []

  # Reading the top 3 characters
  with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Top 10 characters/" + title + " - Top 10 Characters.txt", 'r') as f:
    for line in f:
      character = line.strip().split(",")[0]
      top_10_characters.append(character)

  top_3_characters = top_10_characters.copy()

  # Reading the top 3 characters in each book
  if title == "The Road to Oz":
    top_3_characters = []
    top_3_characters.append(top_10_characters[0])
    top_3_characters.append(top_10_characters[2])
    top_3_characters.append(top_10_characters[3])

  elif title == "The Scarecrow of Oz":
    top_3_characters = top_3_characters[0:2]
    top_3_characters.append(top_10_characters[3])

  else:
    top_3_characters = top_3_characters[0:3]

  # Reading the Emotion Arc
  with open(path2, "r") as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
      if len(row) >= 2:
        X1.append(row[0])
        emotional_arc.append(row[1])

  X1 = X1[1:]
  emotional_arc = emotional_arc[1:]
  X1 = [int(x) for x in X1]
  emotional_arc = [float(x) for x in emotional_arc]
  X1 = [x for x, score in zip(X1, emotional_arc) if score != 0]
  emotional_arc = [score for score in emotional_arc if score != 0]

  for character in top_3_characters:

    # Reading the Character Arc
    filename = path1 + title + " - " + character + ".csv"
    character_score = []
    X = []

    with open(filename, "r") as csvfile:
      csvreader = csv.reader(csvfile)
      for row in csvreader:
        if len(row) >= 2:
          X.append(row[0])
          character_score.append(row[1])

    X = X[1:]
    character_score = character_score[1:]
    X = [int(x) for x in X]
    character_score = [float(x) for x in character_score]
    X = [x for x, score in zip(X, character_score) if score != 0]
    character_score = [score for score in character_score if score != 0]

    fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 5))

    axes[0].plot(X, character_score)
    axes[0].set_xlabel("Percentage of Book")
    axes[0].set_ylabel("Emotional Score based on Valence")
    axes[0].set_title("Character Arc - " + character)

    axes[1].plot(X1, emotional_arc)
    axes[1].set_xlabel("Percentage of Book")
    axes[1].set_ylabel("Emotional Score based on Valence")
    axes[1].set_title("Emotion Arc of " + title)

    fig.suptitle(title + " - " + character, fontsize = 32)
    plt.tight_layout()
    plt.savefig("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Results - Plots/Character and Emotion Arc Comparison/" + title + "/" + title + " - " + character + ".pdf")
    plt.show()
    print()
    print()

# 21. Main Function:

In [ ]:
path = "/content/drive/MyDrive/The Wizard of Oz"

# Getting all the books in the Folder
for root, _, files in os.walk(path):
    for file in files:
        if file != ".DS_Store":

          b = os.path.join(root, file)

          # Opening the books using codecs
          book = codecs.open(b, "r", encoding = "utf-8")

          # Getting the Title of the book
          title = get_title(book)

          # Removing the uneccesary information related to Project Gutenberg
          # at the beginning and end of the book
          book = codecs.open(b, "r", encoding = "utf-8")
          book_clean = clean_end_first(book, file)

          # Getting all the chapter titles from the book in order
          chapter_title = chapter_titles(book_clean, file)

          # Getting the text associated with each chapter in order
          chapter_chuncks = chapter_chunck(chapter_title, book_clean, file)

          # Removing paragraph spacing and line breaks
          chapter_chuncks_clean = remove_spacing_line_breaks(chapter_chuncks)

          # Sentence Tokenisation for each chapter in book
          all_sents_in_each_chapter = sent_tokenise(chapter_chuncks_clean)

          # Getting all the words in the whole text
          all_sents_list = list(itertools.chain(*all_sents_in_each_chapter))

          # Named Entity Recognition using AllenNLP and the results will be stored in a CSV file
          # Characters name and the sentence ID
          PER_tags = NER(all_sents_list, title)

          # Reading the Named Entity Recognition CSV file
          # List of all appearances of the name
          list_of_names = NER_read_csv(title)

          # Making corrections to some of the NER model tags
          list_of_names_corrected = correction_function(list_of_names, title)

          # Using the NameParser to get the titles of all the characters in the novel
          # This includes the following information: title, firstname, middlename, lastname, suffix and nickname
          name_info = get_name_info(list_of_names_corrected)

          # Getting all the information together in a single list
          total_names_info = []

          for i in range(len(list_of_names_corrected)):
            d1 = list_of_names_corrected[i]
            d2 = name_info[i]
            d3 = {"gender": ""}

            d4 = {**d1, **d2, **d3}
            total_names_info.append(d4)

          # Getting the gender of the characters
          total_names_info = gender_identification(total_names_info, title)

          # Removing the characters with no gender or unknown genders
          total_names_info_corrected = []

          for i in range(len(total_names_info)):
            if total_names_info[i]["gender"] != "unknown" and total_names_info[i]["gender"] != "":
              total_names_info_corrected.append(total_names_info[i])

          # Storing all the information in a CSV file
          store_all_info(total_names_info_corrected, title)

          # Getting the characters in the book
          characters = get_characters(total_names_info_corrected)

          # Saving the characters in the novel in a text file
          with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Characters/" + title + " - Characters.txt", 'w') as f:
            for character in characters:
              f.write(str(character) + '\n')

          # Matching Algorithm
          all_entities = matching_algorithm(title)

          # Reading nickname information from the text file
          unique_entities = nicknames_and_matches(title, all_entities)

          # Storing the matching entities in a text file
          with open("/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Matched Characters and Nicknames/" + title + " - Matches.txt", 'w') as f:
            for i, entity in enumerate(unique_entities):
              line = ','.join(entity) + '\n'
              f.write(line)

          # Entity Resolution and getting the top 10 characters based on their occurence
          entity_resolution(title)
          top_10_characters(title)
          resolution(title, all_sents_list)

          # Coreference Resolution
          coreference_resolution(title)

          # Semantic Role Labelling
          semantic_role_labelling_coref(title)

          character_based_sentences_coref(title)

          # Character Arc Generation
          emotional_arc(title, "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Character Text for Character Arc Generation - Coref/", "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Character Arc - Coref/")

          # Comparing the character arcs for each book
          path5 = "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Character Arc - Coref/" + title + "/"
          path6 = "/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/" + title + "/" + title + ".csv"
          character_arc_compare(title, path5, path6)

          # Plotting the individual character arcs
          path7 = "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Character Arc - Coref/" + title + "/"
          individual_arcs(path7, title)

          # Comparing individual character arcs with the emotion arc of the narrative
          path8 = "/content/drive/MyDrive/The Wizard of Oz/Character Arc - Trial 1/Final Outcomes/Character Arc - Coref/" + title + "/"
          path9 = "/content/drive/MyDrive/The Wizard of Oz/Emotional Arc - Trial 1/" + title + "/" + title + ".csv"
          character_and_emotional_comparison_individual(title, path8, path9)